# The Problem

From an interesting queston posed on stackoverflow:

https://stackoverflow.com/questions/53489058/improved-efficiency-versus-iterating-over-two-large-pandas-dataframes/

The premise:
With two large arrays of lat-lng coordinates, obtain the number of points a coordinate in the first array is "close" (within 1 km) to in the second array. So if we have two arrays (_n x 2_ and _k x 2_ where _n > k_), the resulting array should be an array of counts of dimension $n$, representing the number of times a point in the first $n$ array is within 1 km of a point within the $k$ array.

However, in the case of the question posed, $n = 10,000,000$ and $k = 1,000,000$. On a naive basis, you would have to calculate $10,000,000 x 1,000,000$ or $10,000,000,000,000$ distances. Unless you can accomplish this on a nanosecond timescale, this task would take a very long time (by the question poser's own estimate, some 6000 hours).

This notebook will go over several approaches:
* A naive serial approach using `geopy`: Easy to program, probably your first pass at solving this problem
* A multiprocessing version of the serial approach
* Parallelized Numba on CPU: Requires knowledge of `Numpy`, `Numba`, and vectorizing code, but fairly straightforward
* Vectorized Numba-CUDA: Requires the above, plus writing `guvectorize` functions to handle complicated array-to-array calculations, and working with CUDA limitations (no `Numpy` typically)
* Numba-CUDA Custom Kernel: Requires managing host-to-gpu memory transfers, managing the CUDA grid/thread schema, and working with the CUDA limitations
 
 Also, [check out this accompanying blog post](https://medium.com/@ernestk.social/better-parallelization-with-numba-3a41ca69452e)

In [1]:
import math
import numpy as np

from numba import cuda, jit, prange, vectorize, guvectorize
from sys import getsizeof
from multiprocessing import cpu_count, Pool

## Initializing Random Data

In [2]:
n = 1_000
k = 1_000

coord1 = np.zeros((n, 2), dtype=np.float32)
coord2 = np.zeros((k, 2), dtype=np.float32)

coord1[:,0] = np.random.uniform(-90, 90, n).astype(np.float32)
coord1[:,1] = np.random.uniform(-180, 180, n).astype(np.float32)
coord2[:,0] = np.random.uniform(-90, 90, k).astype(np.float32)
coord2[:,1] = np.random.uniform(-180, 180, k).astype(np.float32)

coord1 = np.sort(coord1,axis=0)
coord2 = np.sort(coord2,axis=0)

## Naive Python

In this version, I go about solving the problem as if it was my first time solving this type of geo-spatial problem. Often, particularly in a new domain, I find that I'll lean heavily on state-of-the-art libraries, stackoverflow, and other prior work in the area.

The following implementation utilizes two simple loops and the latitude filter mentioned above. Additionally, it leans on geopy for a distance function (`great_circle` which is akin to haversine distance). The `great_circle` implementation is much quicker than `geodesic`, but we'll see later that it doesn't hold up to a numba-accelerated function.

Although I didn't initally, I will be using the latitude filter that was suggested and implemented in the stackoverflow post. The distance between latitudes (straight north-south) maps pretty consistently to actual distance in km (slightly varies since the Earth is not a perfect sphere). By picking the distance criterion and dividing it by 100, it should eliminate those points (based on latitude alone) that would too far away to be worth calculating the actual distance.

In [3]:
from geopy.distance import geodesic, great_circle

In [4]:
%timeit geodesic(coord1[0], coord2[0])

169 µs ± 408 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
%timeit great_circle(coord1[0], coord2[0])

14 µs ± 47.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


The time comparison shows a 10x increase in speed! In theory, we should be able to process about 200 million distances per hour with this function. Unfortuanately that still falls short of the 10 trillion distances we may need to process for the full coordinate set.

In [6]:
def get_nearby_py(coord1, coord2, max_dist):
    out = []
    lat_filter = max_dist / 100
    for lat,lng in coord1:
        ct = 0
        for lat2,lng2 in coord2:
            if np.abs(lat - lat2) < lat_filter:
                if great_circle((lat,lng),(lat2,lng2)).km < max_dist:
                    ct += 1
        out.append(ct)
    return out

In [7]:
_t_py = %timeit -o get_nearby_py(coord1, coord2, 1.0)

3.49 s ± 41.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
# Estimate of time for full data set (hours)
est_time = lambda x : print( '{} Est. Hours to Process the Full Data Set'.format( round(1e13/(n*k) * x / 3600, 2) ) )
est_time(_t_py.average)

9698.21 Est. Hours to Process the Full Data Set


## Multiprocessing Python

We use the `starmap` function within `Pool` in order to pass it args. Unfortunately, we need to create a tuple of args for for each instance of `get_nearby_py` that will be created (12 in this case). 

The list comprehension includes tuple each with a portion of coord1, the full coord2, and the distance metric.

We observe about a 6x speed up (correpsonding to physical cores), but that's not nearly good enough for this challenge.

In [9]:
def get_nearby_mp(coord1, coord2, max_dist):
    cores = cpu_count()
    coord1_split = np.array_split(coord1, cores)
    starmap_args = [(split, coord2, 1.0) for split in coord1_split]
    with Pool(cores) as p:
        data = np.concatenate(p.starmap(get_nearby_py, starmap_args))

In [10]:
_t_mp = %timeit -o get_nearby_mp(coord1, coord2, 1.0)
est_time(_t_mp.average)

407 ms ± 51.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1131.13 Est. Hours to Process the Full Data Set


## Numba-CPU Performance

My intuition was that geopy was doing fancy things that make it a great general purpose library for geospatial analysis, but not for the brute force computations we require at the moment. My first step was to implement haversine distance to boost the distance calculation portion of solving this problem.

The haversine function alone represents a near 15x improvement over `geopy.great_circle()`, allowing us to calculate 3 billion distances per hour.

In [11]:
@jit(nopython=True)
def haversine(s_lat,s_lng,e_lat,e_lng):
    # approximate radius of earth in km
    R = 6373.0

    s_lat = np.deg2rad(s_lat)                    
    s_lng = np.deg2rad(s_lng)     
    e_lat = np.deg2rad(e_lat)                       
    e_lng = np.deg2rad(e_lng)  

    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2

    return 2 * R * np.arcsin(np.sqrt(d))

In [12]:
_t_hav = %timeit -o haversine(coord1[0,0],coord1[0,1],coord2[0,0],coord2[0,1])
print(int(3600 / _t_hav.average))

897 ns ± 10.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
4015500947


Next, I utilized Numba to parallelize the search across the arrays. Here `prange` and the decorator `@jit(nopython=True, parallel=True)` run each loop of `for i in range(n)` as an independent process. While this will lead to a substantial speed increase, this could also lead to race conditions or cause issues for inter-loop dependencies. Fortunately, the steps taken in this function have no such dependencies (and each loop is writing to its own space in the output).

In [13]:
@jit(nopython=True, parallel=True)
def get_nearby_numba(coord1, coord2, max_dist):
    '''
    Input: `coords`: List of coordinates, lat-lngs in an n x 2 array
           `coords2`: List of port coordinates, lat-lngs in an k x 2 array
           `max_dist`: Max distance to be considered nearby
    Output: Array of length n with a count of coords nearby coords2
    '''
    # initialize
    n = coord1.shape[0]
    k = coord2.shape[0]
    output = np.zeros(n, dtype=np.int32)
    lat_filter = max_dist / 100
    
    # prange is an explicit parallel loop, use when operations are independent and no race conditions exist
    for i in prange(n):
        # comparing a point in coord1 to the arrays in coord2
        point = coord1[i]
        # subsetting coord2 to reduce haversine calc time.
        coord2_filtered = coord2[np.abs(point[0] - coord2[:,0]) < lat_filter]
        # in case of no matches
        if coord2_filtered.shape[0] == 0: continue
        # returns an array of length k
        dist = haversine(point[0], point[1], coord2_filtered[:,0], coord2_filtered[:,1])
        # sum the boolean of distances less than the max allowable
        output[i] = np.sum(dist < max_dist)

    return output

In [14]:
_t_ncpu = %timeit -o get_nearby_numba(coord1, coord2, 1.0)
est_time(_t_ncpu.average)

480 µs ± 28.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.33 Est. Hours to Process the Full Data Set


## Numba-CUDA guFunc Performance

The `vectorize`, `guvectorize`, and `cuda.jit` decorators allow us to write functions that will be run on our GPU. However, they differ significantly between each other.

`vectorize` is useful for functions that rely on straight-forward applications of broadcasting. Think functions of the form, in terms of input and output: (m) `+-/*` (1) -> (m)

`guvectorize` is useful for more complicated arrays where broadcasting may not be obvious. For example, imagine trying to find the sum of two arrays: (m) + (n) -> (?). This may be interpreted several ways:
- (m) + (n) -> (m), Here the resulting array may represent each elemet of m summed with every element of n.
- (m) + (n) -> (n), Like above, except each element of n summed with elements of m.
- (m) + (n) -> (), A reduction operation to a scalar, basically the elements of m and n summed together.
- You can probably think of many more.

With `guvectorize`, you'll often need to write loops to iterate across the dimensions of an array to arrive at your explicit intention. Additionally, the output array or scalar must be passed as a parameter and mutated (no return statement).

`cuda.jit` is perhaps the most complicated, as you'll be utilizing the CUDA design pattern for programs. This will be discussed at a later section, but for now, know that `cuda.jit` allows for the creation of _kernels_ that are the most flexible way to program for a GPU.

In [18]:
@cuda.jit(device=True)
def haversine_cuda(s_lat,s_lng,e_lat,e_lng):
    '''
    This is now a non-vectorized version of the haversine distance function. 
    All inputs are expected to be scalars.
    '''
    # approximate radius of earth in km
    R = 6373.0

    s_lat = s_lat * math.pi / 180                     
    s_lng = s_lng * math.pi / 180 
    e_lat = e_lat * math.pi / 180                    
    e_lng = e_lng * math.pi / 180 

    d = math.sin((e_lat - s_lat)/2)**2 + math.cos(s_lat)*math.cos(e_lat) * math.sin((e_lng - s_lng)/2)**2

    return 2 * R * math.asin(math.sqrt(d))

@guvectorize(['(float32[:,:], float32[:,:], float32, int32[:])'], '(m,w),(n,w),() -> (m)', target='cuda')
def get_nearby_ufunc(coord1, coord2, max_dist, out):
    n = coord1.shape[0]
    k = coord2.shape[0]
    lat_filter = max_dist / 100
    
    # will need to manually iterate through the two coord sets
    for i in range(n):
        ct = 0
        for j in range(k):
            # escape condition if the point in coord2 is not relevant (too far latitude-wise)
            if math.fabs(coord1[i,0] - coord2[j,0]) > lat_filter: continue
            dist = haversine_cuda(coord1[i,0], coord1[i,1], coord2[j,0], coord2[j,1])
            if dist < max_dist: ct += 1
        out[i] = ct

In [16]:
out = np.empty(n, dtype=np.int32)
_t_nguv = %timeit -o get_nearby_ufunc(coord1, coord2, 1.0, out)
est_time(_t_nguv.average)

105 ms ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
292.27 Est. Hours to Process the Full Data Set


You may notice that this seems to be a big step back in terms of performance. Part of the problem is that we're not utilizing the full potential of the hardware through a _kernel_. However, we're also not utilizing the memory on the GPU with this current pattern. Instead, we're forcing many host-to-GPU transfers that are killing performance.

## Numba-CUDA Kernel Performance - Global Memory

We're doing a few things differently (although the working code is mostly the same) so we'll cover the changes to memory and the kernel in two parts:

#### Global Memory

With just a few tweaks to how we store the arrays and variables, we can reduce the number of shuffles between host and GPU and boost performance. However, if we want to use the result of this function, we'll need to move `out_gpu` back from the device.

You may also be wondering why I cast the arrays to 32-bit precision. I'm currently running this on an RTX 2070. While a powerful GPU, this model does not have strong support for [double precision (64-bit) calculations.](https://en.wikipedia.org/wiki/GeForce_20_series)

#### Kernel

We need to define the [grid, block, thread structure of our kernel](https://people.maths.ox.ac.uk/gilesm/cuda/lecs/lec1.pdf). Additionally, we have to tell the gpu how we're going to iterate over these structures while completing this work. Fortunately, Numba exposes a few helper functions like `cuda.grid()` and `cuda.gridsize()`.

Basically, the first for-loop will iterate through each thread within each block, then continue on to the next block. However, we won't need to manually keep track of CUDA's numbering scheme. Instead, we'll be iterating through all of the blocks and threads but we'll get to treat i as ranging from `0, 1, 2, 3,..., End_of_the_array`. Since we know `out` will be of equal size to `coord1`, we can write directly to `out[i]` without worrying about simultaneousy read/writes. 

In other words, each thread will manage one element of `out`, iterate through the `coord2`, and mutate it's single element until the entire span of `coord1` is exhausted. This behaviour prevents us from having to write some kind of data-loader, which is awesome. These threads will execute in blocks of 1024, 36 grids at a time. These numbers were picked due to the number of stream processors available on my particular hardware, and trial-and-error.

In [15]:
@cuda.jit
def get_nearby_kernel(coord1, coord2, max_dist, out):
    start = cuda.grid(1)
    stride = cuda.gridsize(1)
    lat_filter = max_dist / 100
    
    for i in range(start, coord1.shape[0], stride):
        ct = 0
        _lat1 = coord1[i,0]
        _lng1 = coord1[i,1]
        
        for j in range(coord2.shape[0]):
            _lat2 = coord2[j,0]
            _lng2 = coord2[j,1]
            # escape condition if latitudes are too far apart
            if math.fabs(_lat1 - _lat2) <= lat_filter:
                dist = haversine_cuda(_lat1, _lng1, _lat2, _lng2)
                if dist < max_dist:ct += 1
                
        out[i] = ct
        
threads_per_block = 512
blocks_per_grid = 36

In [16]:
coord1_gpu = cuda.to_device(coord1)
coord2_gpu = cuda.to_device(coord2)
out_gpu = cuda.device_array(shape=(n,), dtype=np.int32)

In [19]:
_t_nker = %timeit -o \
get_nearby_kernel[blocks_per_grid, threads_per_block](coord1_gpu, coord2_gpu, 1.0, out_gpu); \
out_gpu.copy_to_host()

730 µs ± 46 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
est_time(_t_nker.average)

2.03 Est. Hours to Process the Full Data Set


This result seems barely better than the Numba-CPU function but I'm betting that the overhead introduced by the GPU is clouding things here, let's see how it does with a bigger data set:

## The Great Test!

Now we pit Numba-CPU vs Numba-GPU on a much larger dataset:

In [21]:
# Initial Data
n = 10_000_00
k = 1_000_000

coord1 = np.zeros((n, 2), dtype=np.float32)
coord2 = np.zeros((k, 2), dtype=np.float32)

coord1[:,0] = np.random.uniform(-90, 90, n).astype(np.float32)
coord1[:,1] = np.random.uniform(-180, 180, n).astype(np.float32)
coord2[:,0] = np.random.uniform(-90, 90, k).astype(np.float32)
coord2[:,1] = np.random.uniform(-180, 180, k).astype(np.float32)

coord1 = np.sort(coord1,axis=0)
coord2 = np.sort(coord2,axis=0)

In [22]:
# CUDA Arrays
coord1_gpu = cuda.to_device(coord1)
coord2_gpu = cuda.to_device(coord2)
out_gpu = cuda.device_array(shape=(n,), dtype=np.int32)

In [23]:
%time cpu_solution = get_nearby_numba(coord1, coord2, 1.0)

CPU times: user 49min 12s, sys: 1.62 s, total: 49min 14s
Wall time: 2min 3s


In [24]:
%%time
get_nearby_kernel[blocks_per_grid, threads_per_block](coord1_gpu, coord2_gpu, 1.0, out_gpu)
# lazy evaluation, calling copy_to_host() will actually execute
gpu_solution = out_gpu.copy_to_host()

CPU times: user 11.5 s, sys: 5.87 s, total: 17.4 s
Wall time: 16.9 s


## Conclusion

Hooray, an RTX 2070 is about 16x faster at this task than a Ryzen 1600 (6-Core, 12-Threads). That's a nice little speed up for what was a first pass. The secret to getting insane speed-ups out of CUDA Kernels is to leverage **shared memory** (a high-speed cache per grid) as much as possible. However, for this problem, the two coordinate sets would exceed the limits of this cache. I'm certain that with a different grid/thread layout, I could have used shared memory, but this example wasn't too difficult to code while yielding a substantial gain in speed.

### Addendum: Arriving at a Different Solution

I noticed a weird behavior that can be summarized below:

In [25]:
print('All Match? ', np.all(cpu_solution == gpu_solution) )
print('No. of Mismatches: ', np.sum(cpu_solution != gpu_solution))

All Match?  False
No. of Mismatches:  2467


In [26]:
mismatch_mask = cpu_solution != gpu_solution
amt_off = cpu_solution[mismatch_mask] - gpu_solution[mismatch_mask]

In [27]:
print(np.mean(amt_off))
print(np.min(amt_off), np.max(amt_off))

-0.024726388325901905
-3 2


Apparently, the cpu_solution was slightly off from the gpu_solution. I'm willing to chalk this up to floating point imprecision. Additionally, although they're functionally the same, the two different haversine functions rely on different libraries (_cpu : numpy_ vs _gpu : math_). Given that there are 10 trillion coordinate-pairs being compared, I can live with less than .001% of mismatches.

I may look further into this issue at a later time.